# Urban Computing - Lab 1
This lab introduces Python for scientific programming, shows you examples on how to handle API access and visualization for location information, how to generate simulated trajectory data, as well as a number of tips for optimizing your code. We assume prior familiarity with numpy and matplotlib. The material in this lab serves as an important basis for your assignments. Therefore, make sure to familiarize yourself with them before the first assignment is due. This lab is not going to be graded and we will make the solutions available later.

This lab contains four parts: 

- Part I Numpy and basic scientific programming
- Part II Maps: OpenRoute Service and Folium
- Part III Foursquare API and Pandas dataframes
- Part IV Troubleshooting poor performance in Pandas (and Numpy)   

But before we start with the execersises here is some information on how to write clean code in Python:
## Code style
Unlike for example C++ where people just can't agree exactly what the best way is to place brackets, for Python there is just one key standard for how good clean code looks like: PEP8. PEP stands for Python Enhancement Proposal. You can find the official description [here](https://www.python.org/dev/peps/pep-0008/). A bit lighter reading version that gives more explanation is [here](https://realpython.com/python-pep8/).

Writing clean code is important. It's very common that you find yourself doing some project and thinking, "hey, I solved this little problem in another problem two months ago, how did I do that again?" If you wrote nice clean code, it's easy to look up how you did it. Likewise, if you're working together with people, it's easier if you understand what others are doing. Adhering (sensibly) to PEP8 makes this easy.

The original PEP8 standard assumes a 80-character line limit, but with modern screens, it's reasonable to assume a 120 character line limit.

Python also has a very definite philosophy behind it, and knowing that philosophy will make it easier to understand how a lot of things are designed in Python. It's called The Zen of Python, officially documented as PEP20, and can be found [here](https://www.python.org/dev/peps/pep-0020/) and there are many essays explaining these 19 statements, for example [here](https://inventwithpython.com/blog/2018/08/17/the-zen-of-python-explained/). Note though that every article explaining them is subtly different. They're intended to make you think, not blindly follow :)

## Part I  Numpy and basic scientific programming

#### Loop Plot
The code below constructs a live-plot visualization inside a for-loop. <br>
It gradually draws $x$,$y$ data points derived from the following trigonometric equations: $$x(t)=t-1.6\cdot cos(24t)\\y(t)=t-1.6\cdot sin(25t)\\ \text{where}\ t \in \{0.01, 0.02, 0.03, ..., 0.18\}$$
Try to change some of the parameters (e.g. `hi_b`, `lo_b`,...) and see the differences.  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def fx(t):
    return t - 1.6 * np.cos(24 * t)


def fy(t):
    return t - 1.6 * np.sin(25 * t)


# Step Parameters for vector t
hi_b = 0.18
lo_b = 0.01
by = 0.02
step = np.arange(lo_b, hi_b, by)
step = np.flip(step, axis=0)

# Slightly perturb
step = np.array([st + ((np.random.rand() - 0.5)/100) for st in step])

# Live-Plot inside the for-loop
for st in step:
    print(st)
    t = np.arange(0, 10, st)
    x = fx(t)
    y = fy(t)
    plt.figure()
    plt.plot(x, y)
    plt.pause(0.4)
plt.show()


### Exercise 1 - Plots
Use the previous information and visualize your own live-plot using the following trigonometric equations:
$$x(u) = sin(33u)cos(9u)\\
y(u) = sin(40u)sin(7u)$$
A correct final image, if everything was written correctly, is something like the following:

<img src="data/plt.png">

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Function for generating x(u)
# Code here:
def fx(u):
    return np.sin(33*u) * np.cos(9*u)


# Function for generating y(u)
# Code here:
def fy(u):
    return np.sin(40*u) * np.sin(7*u)


# Generate vector u
hi_b = 0.018
lo_b = 0.001
by = 0.002
step = np.arange(lo_b, hi_b, by)
step = np.flip(step, axis=0)

# # Slightly perturb
# step = np.array([st + ((np.random.rand() - 0.5)/100) for st in step])

# Live-Plot inside the for-loop
plt.figure()
for st in step:
    t = np.arange(-1, 1, st)
    x = fx(t)
    y = fy(t)
    plt.plot(x, y, color='tab:blue')
    plt.plot(x, -y, color='tab:blue')
    # plt.pause(0.3)
plt.show()


## Part II  Maps: OpenRoute Service and Folium 

#### Download APIs for Python
- #### Some basic API's: 
    - The `openrouteservice` [(documentation)](https://openrouteservice-py.readthedocs.io/en/latest/) library is already installed in the development container. The current package provides a functional python-API for requesting geographical data directly from openrouteservice. You will have to create an account and get the API_KEY in order to be able to work with this library. You can find more information in the website [openrouteservice.org](https://openrouteservice.org/services/) where you can also create such an account, read the documentation and play around with online data request forms.

    - The second library is called `folium` [(documentation)](https://python-visualization.github.io/folium/) and it provides API for map visualization. It has already been installed in the development container.

 - #### Using `openrouteservice`
     - Import the library, provide some coordinates and request the route. 
     - Try to understand the content of the routes dictionary.
     - Note that we always have to provide the api_key when we generate a client.

In [ ]:
import openrouteservice as opn
from openrouteservice.directions import directions

api_key = "5b3ce3597851110001cf62482fff3e10b7da47968742a37590c03b6d"
coords = ((8.34234, 48.23424), (8.34423, 48.26424))

client = opn.Client(key=api_key)  # Specify your personal API key
routes = directions(client, coords)
routes


- #### Decode a PolyLine
The output of the `directions` function is a json file by default. We can decode it to a simple dictionary which, in this case, is a GeoJSON geometry. We can simply do the following:

In [ ]:
import openrouteservice as opn
from openrouteservice.directions import directions
from openrouteservice import convert

api_key = "5b3ce3597851110001cf62482fff3e10b7da47968742a37590c03b6d"
coords = ((8.34234, 48.23424), (8.34423, 48.26424))

client = opn.Client(key=api_key)  # Specify your personal API key

# decode_polyline needs the geometry only
geometry = client.directions(coords)["routes"][0]["geometry"]

decoded = convert.decode_polyline(geometry)

decoded  # this has only the coordinates for the route.


- #### Visualize the maps
    - Import `folium` library
    - Get familiar using the [link](https://python-visualization.github.io/folium/quickstart.html)
    - Try to plot a map using the the coordinates: 52.169709,4.457111 
    - Is the place on the map familiar to you?
    

In [ ]:
import folium

# Set up folium map
map1 = folium.Map(location=([52.169709, 4.457111]), zoom_start=17)
map1


- #### Visualize specific locations
    1. Plot the map around the previous location.
    2. Specify university buildings.
    3. Request isochrones within some minutes walking from each building.
    4. Draw the radius of isochrones (note the reversed coordinates).
    

In [ ]:
import openrouteservice as opn
import folium

# Specify your personal API key
api_key = "5b3ce3597851110001cf62482fff3e10b7da47968742a37590c03b6d"
clnt = opn.Client(key=api_key)

# 1. Map
map1 = folium.Map(location=([52.169709, 4.457111]), zoom_start=16)

# 2. Set up the building dictionary with real coordinates
bld_dict = {
    "Archeologie": {"location": [4.464366, 52.165078]},
    "Gorlaeus": {"location": [4.459992, 52.167412]},
    "Jansen": {"location": [4.455754, 52.167730]},
    "Chemistry": {"location": [4.459306, 52.168101]},
}

# 3. Request of isochrones with 2 minute footwalk.
params_iso = {
    "profile": "cycling-road",
    "intervals": [120],  # 120/60 = 2 mins
    "segments": 120,
    "attributes": ["total_pop"],  # Get population count for isochrones
}

# 4. Add the isochrones to the map and visualize it
for name, bld in bld_dict.items():
    # Add buildings coords to request parameters
    params_iso["locations"] = [bld["location"]]

    # Perform isochrone request
    bld["iso"] = clnt.isochrones(**params_iso)
    folium.features.GeoJson(bld["iso"]).add_to(map1)

    # Create a map icon
    icon = folium.Icon(
        color="lightgray",
        icon_color="#cc0000",
        icon="home",
        prefix="fa",
    )

    # reverse coords due to folium syntax:Latitude/Longitude of Map (Northing, Easting).
    # Add marker to map.
    reversed_coords = list(reversed(bld["location"]))
    marker = folium.map.Marker(reversed_coords, icon=icon, popup=name)
    marker.add_to(map1)

map1


- #### Simulating trajectories of horseback riders
Near LIACS, on the Wassenaarseweg, there are stables where children go for horseback riding. Here, we would like to simulate the daily rounds that the horses do. Such simulations can help when assessing how robust algorithms are to noise or how well they perform in general. We will use simulated trajectories in the first assignment.
While there are other ways to get an object with timestamped longitudes and latitudes, we are presenting here a basic version. Feel free to increase the difficulty or add more cycles (i.e. additional daily or weekly trajectories) as you wish.

The only movement that the horses do is one round in the park, the rest of the time, they stay in the stables. There are two locations in the park where the children will take a break and have a snack for 30 min. Try to generate and visualize these simulated trajectories.


In [ ]:
import openrouteservice as opn
from openrouteservice.directions import directions
from openrouteservice import convert
import numpy as np
import folium
import time


# A) coordinates of the stables and the two picnic sites:

locations = dict(
    ridingschool = [4.467626, 52.170207],
    corner_one = [4.474798, 52.172679],
    corner_two = [4.475071, 52.169961]
)


# B) Get the circuit from open route services:

api_key = "5b3ce3597851110001cf62482fff3e10b7da47968742a37590c03b6d"
client = opn.Client(key=api_key)


# C) Create an object with timestamped long/lat
# Say it takes 20 min (the children are very slow) to get from one fix point to the next:
# Remember that the break times are 30 min long
# Fuse timestamps, long, lat into one object

legs = [
    {'start_c': locations['ridingschool'], 'end_c': locations['corner_one'], 'trajectory': [], 'timestamps': []},
    {'start_c': locations['corner_one'], 'end_c': locations['corner_two'], 'trajectory': [], 'timestamps': []},
    {'start_c': locations['corner_two'], 'end_c': locations['ridingschool'], 'trajectory': [], 'timestamps': []},
]

routes = {}

for i, leg in enumerate(legs):

    geometry = client.directions(
        (leg['start_c'], leg['end_c']),
        profile = 'foot-walking'
    )["routes"][0]["geometry"]
    trajectory = convert.decode_polyline(geometry)['coordinates']
    timestamps = np.arange(i*50, i*50+20, 20/(len(trajectory)+1))[1:]
    timestamps = [time.strftime('%H:%M:%S', time.gmtime(ts*60)) for ts in timestamps]
    leg.update({'trajectory': trajectory, 'timestamps': timestamps})


# D) Fill the rest of the day with "Staying at stables" ???


# E) Can you visualize the trajectory?
# Set up folium map

center = np.average([coords for coords in locations.values()], axis = 0)
map1 = folium.Map(list(reversed(center)), zoom_start = 16)



for leg in legs:
    
    for coords, timestamp in zip(leg['trajectory'], leg['timestamps']):
        icon = folium.Icon(
            color = 'lightgray',
            icon_color = '#cc0000',
            icon = 'map-pin',
            prefix = 'fa'
        )
        marker = folium.map.Marker(list(reversed(coords)), icon = icon, popup = timestamp)
        marker.add_to(map1)

for name, coords in locations.items():
    icon = folium.Icon(
            color = 'blue',
            icon_color = '#cc0000',
            icon = 'home',
            prefix = 'fa'
    )
    marker = folium.map.Marker(list(reversed(coords)), icon = icon, popup = name)
    marker.add_to(map1)

map1


# F) How would you add noise to the trajectory? What kind of noise could occur?


## Part III  Foursquare API and pandas dataframes

- ####  Foursquare API
Visit [foursquare](https://developer.foursquare.com/places-api) site and create a developer free account. Start an app and install [foursquare-python-API](https://pypi.org/project/foursquare/) by typing on a terminal: ``` pip install foursquare```, or download it from the site. 

The following block of code provides some information about how to use `foursquare-API` in python. All you need is to create a developer's account and connect it using the `client_id` and `client_secret` hash from your application in [foursquare](https://developer.foursquare.com/places-api). Then, you create a client object and use it for data request. Please have a look at the [foursquare-documentation](https://developer.foursquare.com/docs).<br>
The specific example requests all bars in specific radius from the Leiden center.

In [ ]:
import foursquare as fsq

# Please copy paste your client_id and client_secret from your web foursquare-app
cl_id = "Use your client_id"
cl_sec = "Use your client_secret"

# Construct the client object
client = fsq.Foursquare(client_id=cl_id, client_secret=cl_sec)

# Make a request
## Request specificaly for venues near the center of Leiden
### ll     --> latitude/longitude
### query  --> type of venue
### intent --> intent performing search
### radius --> Limit results to venues within this
### limit  --> Number of results to return, up to 50
data = client.venues.search(
    params={
        "ll": "52.160236, 4.497012",
        "query": "bar",
        "intent": "browse",
        "radius": 300,
        "limit": 50,
    }
)
print("Bars near my house: " + str(len(data["venues"])))


#### Exercise 2 - Foursquare
#### a. Index the requested dictionary and convert it into a pandas DataFrame
1. Request from foursquare all possible venues within 3000 meters radius of the Leiden city center. Find the coordinates of Leiden Center from google-maps or any other map application.
2. Make a function which takes a dictionary as input.
2. Index the dictionary and get interesting information.
3. Make a DataFrame from that dictionary with the following information in columns: name, latitude, longitude, distance, address, genre (pluralName).
4. You noticed that in many cases some of the information is missing. Make sure that your DataFrame is loaded with the correct information from the dictionary where it was possible, anywhere else fill in with 'NA'. 
5. Return the dataframe.

In [ ]:
import pandas as pd
import requests
import json

url = 'https://api.foursquare.com/v3/places/search'

cl_id = '5QQCPFS2R045Y4HELFZB2BGI3SZ35WTCFUSEBWTGM3IF1NRT'
cl_sec = 'DQPIORSBGEHHRITQPPLKXDSK3PCKGI13GBMEKH5AZK1BJ4YZ'
auth = 'fsq30wbR04nEArbJHJA6Yr4/4pznm4ytwBoIRvSY+wCnpcM='

headers = dict(
    accept = 'application/json',
    authorization = auth
)

params = dict(
    client_id = cl_id,
    client_secret = cl_sec,
    ll = '52.1665,4.4870',
    radius = 3000,
    limit = 50
)

response = requests.get(url, headers=headers, params=params)
data = response.json()

def try_keys(d: dict, keys: list[str]) -> str:
    data = d
    try:
        for key in keys:
            data = data[key]
    except KeyError:
        return 'NA'
    return data

def venue_scraper(data: dict) -> pd.DataFrame:
    results = data['results']
    
    df = pd.DataFrame(columns=['name', 'latitude', 'longitude', 'distance', 'address', 'genre'])
    for i, res in enumerate(data['results']):
        name = res['name']
        latitude = try_keys(res, ['geocodes', 'main', 'latitude'])
        longitude = try_keys(res, ['geocodes', 'main', 'longitude'])
        distance = try_keys(res, ['distance'])
        address = try_keys(res, ['location', 'address'])
        genre = [try_keys(category, ['name']) for category in res['categories']]

        df.loc[i] = [name, latitude, longitude, distance, address, genre]
    return df

venue_scraper(data)

#### b. Count the frequencies of the different venue types (genres)
Search in [pandas-documentation](https://pandas.pydata.org/pandas-docs/stable/reference/index.html) and try to answer the following:
1. Count the frequencies of each venue-type.
2. Count the number of NA values.
3. Find the nearest venue in the center of Leiden.
4. Find the most distant venue from the center.

In [ ]:
venues_df = venue_scraper(data)

# Count frequencies
frequencies = {}
for genres in venues_df['genre']:
    for genre in genres:
        try:
            frequencies[genre] += 1
        except KeyError:
            frequencies[genre] = 1

print("Frequencies of Venue-Types\n\n" + str(x))
for category, frequency in frequencies.items():
    print(category, frequency)

# Find NA and count their frequencies
NA_count = sum([(c=='NA') for col in venues_df for c in venues_df[col]])
print("\nFrequency of NA: " + str(NA_count))

# Nearest venue
near_venue = venues_df.sort_values('distance').iloc[0]['name']
# Distant venue
far_venue = venues_df.sort_values('distance', ascending=False).iloc[0]['name']

print("\nClosest Venue: " + str(near_venue) + "\nDistant Venue: " + str(far_venue))


#### c. Visualize Frequencies 
Plot a barplot for venue-types (genre), using pandas and matplotlib, only for frequencies which are:
$\text{genre}\geq2$. Barplots are suitable graphs for frequencies visualization but this is not the only way. Find another proper way to visualize those frequencies.


In [ ]:
import matplotlib.pyplot as plt

# Query the dataframe for values bigger or equal than 2
frequencies_df = pd.DataFrame(columns=['category', 'frequency'])
for i, (category, frequency) in enumerate(frequencies.items()):
    frequencies_df.loc[i] = [category, frequency]
frequencies_df = frequencies_df.query('frequency>=2').set_index('category')

# First way of plotting frequencies Barplot
bar_ax = frequencies_df.plot.bar(rot=45)
plt.xticks(horizontalalignment='right')

# Second way of plotting frequencies
pie_ax = frequencies_df.plot.pie(y='frequency')
pie_ax.get_legend().remove()
pie_ax.set_ylabel('');


#### d. Visualize locations in maps
Use all previous information and draw pins for all venues in the center of leiden.<br>
Open a map using folium and initialize it with Leiden center coordinates: 52.159536, 4.491366.<br> Then, plot all the venues from the dataframe.

In [ ]:
import folium

# Make a list with the coordinates of each location
locations = {v['name']: (v['longitude'], v['latitude']) for _, v in venues_df.iterrows()}

# Open a map and initialize it with the coordinates
map2 = folium.Map(location=[52.159536, 4.491366], zoom_start=15)

for name, loc in locations.items():

    # Create a map icon
    icon = folium.Icon(
        color="lightgray",
        icon_color="#cc0000",
        icon="map-pin",
        prefix="fa",
    )

    # reverse coords due to folium syntax:Latitude/Longitude of Map (Northing, Easting).
    # Add marker to map.
    reversed_coords = list(reversed(loc))
    marker = folium.map.Marker(reversed_coords, icon=icon, popup=name)
    marker.add_to(map2)

map2


## Part IV Troubleshooting poor Pandas (and Numpy) performance

### Profiling and Optimizing Loops

As a data scientist, you're going to work with large amounts of data. Numpy, Pandas and Matplotlib are fully capable of doing a lot of work for you, but it matters a lot if you use them in the right way. If your code takes a lot of time to run, then it's hard to work on it, run tests, improve it, and do interesting experiments. This section of the lab focuses on finding the problem spots and speeding them up.

You might think that doing the heavy lifting in a Jupyter Notebook seems a bit strange because usually the more GUI the more performance you sacrifice. However, the cell-based setup of these notebooks actually makes it very easy to zoom in on parts of your program and evaluate their performance, using Jupyter's *magic commands*. We're going to be using two specific commands a lot: `timeit` and `lprun`. `timeit` is already built-in, but `lprun` needs to be installed through `pip`. However, this has already been done in our development container (see also the *"requirements.txt"* file),  To use `lprun` we need to load it.

This section of the tutorial is inspired largely by a [talk](https://www.youtube.com/watch?v=HN5d490_KKk) given at PyCon '17 by Sofia Heisler. You will need a dataset which (if it wasn't bundled with this lab) can be downloaded [here](https://github.com/s-heisler/pycon2017-optimizing-pandas/blob/master/pyCon%20materials/new_york_hotels.csv).

In [ ]:
# Load our tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load the line profiler extension (lprun)
%load_ext line_profiler

In [ ]:
# Load and peek at the dataset
df = pd.read_csv("data/new_york_hotels.csv", encoding="cp1252")
df.head()


We're going to be comparing these hotels by measuring their [great circle distance](https://en.wikipedia.org/wiki/Great-circle_distance) to a constant location using the [Haversine formula](https://en.wikipedia.org/wiki/Haversine_formula). Our constant location will be a shop called [Brooklyn Superhero Supply Co.](https://www.superherosupplies.com/) which is located at (40.671, -73.985).

In [ ]:
# Define a basic Haversine distance formula
def haversine(lat1, lon1, lat2, lon2):
    MILES = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    total_miles = MILES * c
    return total_miles


Let's say that we started out with just a simple loop to collect the distances of all these hotels to the shop;

In [ ]:
# Define a function to manually loop over all rows and return a series of distances
def haversine_looping(df):
    distance_list = []
    for i in range(0, len(df)):
        d = haversine(40.671, -73.985, df.iloc[i]["latitude"], df.iloc[i]["longitude"])
        distance_list.append(d)
    return distance_list


However, our program seems to be taking a long time and we want to figure out if this function is causing the problem. We can check how much time this function takes with the `timeit` magic command. Magic commands are a feature of Jupyter Notebook that you use by putting either a single % in front of them (to apply them to one line) or a double %% to apply them to the whole code cell. So, let's see how long our code is taking:

In [ ]:
# Run the haversine looping function
%timeit df["distance"] = haversine_looping(df)

Is that slow? Well, this is only a very modest dataset (1631 rows), so yeah, actually that's slow. If we were only this dataset to trial our code before deploying it for a bigger problem, then we'd run into trouble. Maybe we can go faster using Pandas' built-in method to iterate over every row, called `iterrows`?

In [ ]:
%%timeit

# Haversine applied on rows via iteration
haversine_series = []
for index, row in df.iterrows():
    haversine_series.append(
        haversine(40.671, -73.985, row["latitude"], row["longitude"])
    )
df["distance"] = haversine_series


That looks about 2.5x as fast, and we didn't have to do anything really complicated. Actually, this code looks a bit nicer. Clearly, how you iterate over a Pandas dataframe matters.

Can we do even better? Yes. There are actually quite a few different methods available, ranging from quick easy fixes to more advanced ones for really heavy lifting. In the next part of this lab we'll show several approaches, but leave it to you to examine which work best.

#### Using `apply` and `lambda` to apply a function to each row
Pandas has another built-in function to apply a function to a part of the dataframe, called `apply`. Apply should be called with an `axis` argument to indicate along which axis to apply the operation. Since we want to apply it to rows, we use `axis=1`.


(The fact that the rows are axis 1, not axis 0, points to an interesting thing: Pandas is in [column major order](https://en.wikipedia.org/wiki/Row-_and_column-major_order), which implies that *any* row-wise traversal is basically going against the grain.) 

In [ ]:
df["distance"] = df.apply(
    lambda row: haversine(40.671, -73.985, row["latitude"], row["longitude"]), axis=1
)


You'll also notice the word `lambda` here. Lamda creates an anonymous function for 'row' that basically wraps our haversine function. We could have also written something like:

In [ ]:
def wrap_haversine(row):
    return haversine(40.671, -73.985, row["latitude"], row["longitude"])


df["distance"] = df.apply(wrap_haversine, axis=1)


Notice that we're writing our function `wrap_haversine(row)` as `wrap_haversine`, so without any argument; that's because what we're actually doing is handing over the `apply_haversine` function itself to Pandas `apply` to apply to each row. 

Why did we even need this `wrap_haversine` function? Because our `haversine` function itself takes four arguments, but we only have one argument (the whole row). So, using `lambda` here is just a bit more convenient than defining a wrapper function.

#### Extracting the columns and looping over them
We could extract the columns and loop over them; maybe that's more efficient than going against the grain of Pandas' column-major ordering?

In [ ]:
lat = df["latitude"]
lon = df["longitude"]
distance_list = []
for i in range(len(lon)):
    distance_list.append(haversine(40.671, -73.985, lat[i], lon[i]))


We haven't changed much from the original naive code - we simplified the lookup of the rows, now we're just using `lat[i]` to get the latitude instead of `df.iloc[i]['latitude']`, and likewise for the longitude. We paid for that with some overhead in extracting the whole column from pandas, but was it worth it?

#### Getting our data column-row instead of row-column
Building on the previous idea, maybe we'd rather not pull out those columns, but what if we just used the same looping idea, looking up columns first and then the row inside the column?

In [ ]:
distance_list = []
for i in range(len(df)):
    distance_list.append(
        haversine(40.671, -73.985, df["latitude"][i], df["longitude"][i])
    )


#### Using the columns as raw Numpy arrays
Pandas is built on Numpy, and you can extract a column as a Numpy array quite easily:

In [ ]:
lon = df["longitude"].values
print(type(lon))


Can we use this to loop faster over it?

In [ ]:
lat = df["latitude"].values
lon = df["longitude"].values
distance_list = []
for i in range(len(lon)):
    distance_list.append(haversine(40.671, -73.985, lat[i], lon[i]))


Why should we expect looping over Numpy arrays to be faster than over a column from a Pandas array (a Pandas Series), or a classic Python list, for that matter?

Pandas Series are built on Numpy arrays, but with some more indexing and other fancy features built in. Extracting the raw Numpy array just skips some of the overhead that comes with that. The difference with Python lists is much more profound. A Python list is allowed to contain any kind of object - floating point numbers, integers, strings, sub-lists, sets, dictionaries, class instances, whole classes, functions, and packages. Clearly, those different kinds of things don't all need the same kind of memory, so they can't be stored in a neat row of equal-size chunks in memory. When you decide to referce item 10 of a list, Python has to look up where item 10 is stored. Meanwhile, a Numpy array is strictly homogenous in its data type - it's all float64, or all integer32 and so forth. Each cell in the array is the same size, so if you know where the array starts, you can easily calculate where each value is. (There is more to it, but this is the key point: Numpy arrays are homogenously typed and that enables optimization tricks.)

### EXERCISE: which other ways work best?
Use `%timeit` with the new methods described above and figure out which ones work best. Which ones look easiest to work with?

#### Vectorized operations
A phrase you'll run into when you dig into Numpy, or if you're familiar with MATLAB, is vectorizing operations. Basically, it means applying an operation across an entire vector of elements all at once, rather than using a loop to process each element separately. Instead of an explicit loop, we use optimized C code built into Numpy that can apply operations to a whole vector all at once.

To see this in action, we're going to use the second magic command we prepared: `lprun`.

In [ ]:
# Haversine applied on rows with line profiler
%lprun -f haversine df.apply(lambda row: haversine(40.671, -73.985, row["latitude"], row["longitude"]), axis=1)

This opens a popup window showing how many times each line in our haversine function has been hit, and how much time has been spend there. We see that we're hitting each line 1631 times, because we're calling the function for each of the 1631 rows in our dataframe. That's clearly some overhead we might be able to get rid of.

Because Pandas Series columns are Numpy arrays under the hood, and because our haversine function uses suitable operations, we're able to easily vectorize our function. All we have to do is call the function with columns as arguments instead of individual values from a row:

In [ ]:
# Vectorized implementation of Haversine applied on Pandas series
%lprun -f haversine df["distance"] = haversine(40.671, -73.985, df["latitude"], df["longitude"])


Looking at the profiling of this version, we now see that we're hitting each line in the function only once. Is it faster?

In [ ]:
%timeit df["distance"] = haversine(40.671, -73.985, df["latitude"], df["longitude"])


Yeah, that's quite a speedup. Of course, the trick we did before, using the raw Numpy values under the Pandas Series, we can combine that:

In [ ]:
%timeit df["distance"] = haversine(40.671, -73.985, df["latitude"].values, df["longitude"].values)

That's even faster. At this point, we should have mostly solved our performance problems, if you can vectorize your operations this neatly, working with Pandas shouldn't be slow at all.

So *if* you can vectorize your functions. What does it take to vectorize a function? Basically, you have to build it using only components that Numpy can apply to a whole vector at a time. For example, the `np.sqrt(x)` function can take the square roots of all the values in a vector all at once.

Vectorizing mathematical operations like addition, subtraction, multiplication, division and so forth is generally straightforward; you might not even realize it but when you write `my_array + my_other_array`, you're using Numpy's addition code which can add two arrays together. For classic linear algebra tasks it's generally a bit more obvious because those transformations have been built as Numpy functions.

A good introductory tutorial is [Look Ma, No For-Loops: Array Programming With NumPy](https://realpython.com/numpy-array-programming/), and a somewhat more advanced tutorial is [“Vectorized” Operations: Optimized Computations on NumPy Arrays](https://www.pythonlikeyoumeanit.com/Module3_IntroducingNumpy/VectorizedOperations.html).

You'll see that vectorization often requires some creative rethinking of your problem, often coming at it sideways. It isn't always necessary to use vectorization, because sometimes the performance hit from a not-too-naive loop is acceptable and rebuilding your function to be vectorized takes too much work. However, if you run into serious performance problems, vectorization can be the answer. 

### Exercise: vectorizing your operations
We start with two vectors of random numbers:

In [ ]:
a = np.random.random(10 ** 7)
b = np.random.random(10 ** 7)


Now we're computing the dot product, that is, the sum of multiplying the elements along the two vectors. This has been coded as follows:

In [ ]:
%%timeit

dot_product = 0
for i in range(len(a)):
    dot_product += a[i] * b[i]


**YOUR TASK** vectorize this code and compare the difference in speed.

In [ ]:
%%timeit
dot_product = np.dot(a, b)


#### Pushing the envelope: optimizing your function line by line
If you've already done vectorization, and performance is still not satisfactory, you can also use the line profiler to look at the lines of your code and see where the most time is spent. For example:

In [ ]:
# Vectorized implementation of Haversine applied on Pandas series
%lprun -f haversine df["distance"] = haversine(40.671, -73.985, df["latitude"].values, df["longitude"].values)

We see that most time is spent in the line `a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2`. If we wanted to try more optimization, that would be were we should focus our efforts.

Doing so is beyond the scope of this tutorial, but it doesn't actually require you to start writing C code. You could use Cython or Numexpr to speed things up, as shown in this tutorial: [Enhancing performance](https://pandas.pydata.org/pandas-docs/stable/user_guide/enhancingperf.html).

### Matplotlib tips and tricks
Consider the code and plot below:

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))

# generate some dummy data
x = np.linspace(-10, 10, 41)  # 41 equally spaced points in [-10, 10]
y = x ** 2

# plot the data
ax.plot(x, y, label="label for my line", color="red")

# make the plot look good
ax.set_title("my title")
ax.set_xlabel("my x label")
ax.set_ylabel("my y label")
ax.legend()  # display the legend; matplotlib tries to put it somewhere convenient
ax.set_xticks(np.linspace(-10, 10, 21))  # set the ticks along the x-axis
ax.set_yscale("log")  # display the y dimension in a log scale
ax.grid()  # turn on the gridlines

# save plots before showing, otherwise you get empty pictures
fig.savefig("plots/A.png")
fig.savefig("plots/B.jpg")
fig.savefig("plots/C.svg")
fig.savefig("plots/D.pdf")
fig.savefig("plots/E.eps")
fig.savefig("plots/F")

# now show it
plt.show()


Consider these questions:

* What is the default file format that Matplotlib saves in?
* Which file format looks good if you enlarge it (especially, if you integrate it in a report)?
* Which file format leaves text as text? This is handy if people want to use copy-paste, or a screen reader for people with disabilities. Or if they still want to edit the graph (e.g., axis labels, sizes, etc.) using another tool.



#### Axes and Figures

You'll also notice that this plot uses a lot of calls on the `ax` object and a few on `fig`, but not a lot of calls to `plt`. In a lot of tutorials you'll see most of the same code, but all calls made to `plt`. Why is that?

Basically, Matplotlib got started as a port of MATLAB plotting capabilities to Python. `plt` is basically a placeholder that points to the most recent figure you were working on. That's fine as long as you're only working on one plot in a project, but it can become confusing if you're generating a lot of pictures.

What we do when we call `fig, ax = plt.subplots()` is generate a new *Figure* with in this case one *Axes* object. The Axes object is what we think of as the actual plot: it has an X and Y axis (hence the name, axes, plural of axis), and it's where we're drawing on. The Figure is more like the windowframe around it. We can have a Figure with multiple Axes in it, which is what happens if you call [plt.subplots](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplots.html) with arguments to indicate how many you'd like; then instead of a single Axes object, you get a bunch of them.  For a more extensive tutorial of "best practices" Matplotlib, I recommend [Python Plotting With Matplotlib (Guide)](https://realpython.com/python-matplotlib-guide/) and [Effectively Using Matplotlib](https://pbpython.com/effective-matplotlib.html).

### Exercise: a larger scale experiment
This is the last task in this lab. Try to think about how you'd set this up if you had to run variants of these experiments many, many times, and you didn't want to have to do everything by hand every time. 

You've already compared a number of optimization techniques, but you've only compared them on one dataset of a specific size. But the method that looks good on this dataset might not scale well to bigger datasets (maybe it's not linear..), so we want to test these methods on multiple sizes of datasets.

**YOUR TASK**
* Choose four different ways of using the haversine function.
* Apply them to this dataset, but scaled up to 2x, 5x, 10, 20x, 50x and 100x the original size.
* Plot the results with Matplotlib. Experiment with the way you set up the plot to get a picture that is useful for analyzing your results.
* Try to find out if the runtime  of any of the methods doesn't linearly scale with the size of the dataset. Does it scale better (i.e. less fast) or worse?

You can enlarge a dataset by just duplicating the rows:

In [ ]:
df2 = df.append([df], ignore_index=True)
df5 = df.append([df] * 4, ignore_index=True)
df10 = df.append([df] * 9, ignore_index=True)
df20 = df.append([df] * 19, ignore_index=True)
df100 = df.append([df] * 99, ignore_index=True)


You can also extract the information we collect by `%timeit` so that you can use it for plotting:

In [ ]:
res = %timeit -o df['distance'] = df.apply(lambda row: haversine(40.671, -73.985, row['latitude'], row['longitude']), axis=1)
runtime = np.mean(res.all_runs)  # measured in seconds
print("mean runtime (s):", runtime)


First we start by defining different functions that apply the haversine function to a dataframe.


In [ ]:
def simple_loop(df: pd.DataFrame) -> pd.Series:
    distances = []
    for i in range(0, len(df)):
        d = haversine(40.671, -73.985, df.iloc[i]['latitude'], df.iloc[i]['longitude'])
        distance_list.append(d)
    return pd.Series(distance_list)

def iterrows_loop(df: pd.DataFrame) -> pd.Series:
    distances = []
    for i, row in df.iterrows():
        d = haversine(40.671, -73.985, row['latitude'], row['longitude'])
        distance_list.append(d)
    return pd.Series(distance_list)

def loop_col_first(df: pd.DataFrame) -> pd.Series:
    distance_list = []
    for i in range(len(df)):
        distance_list.append(haversine(40.671, -73.985, df["latitude"][i], df["longitude"][i]))
    return pd.Series(distance_list)

def apply_lambda(df: pd.DataFrame) -> pd.Series:
    return df.apply(lambda row: haversine(40.671, -73.985, row['latitude'], row['longitude']), axis=1)

def apply_predefined(df: pd.DataFrame) -> pd.Series:
    def haversine_wrapper(row: tuple) -> float:
        return haversine(40.671, -73.985, row['latitude'], row['longitude'])
    return df.apply(haversine_wrapper, axis=1)

def pd_vectorized(df: pd.DataFrame) -> pd.Series:
    return haversine(40.671, -73.985, df["latitude"], df["longitude"])

def np_vectorized(df: pd.DataFrame) -> pd.Series:
    return pd.Series(haversine(40.671, -73.985, df['latitude'].values, df['longitude'].values))


Now we time them.


In [ ]:
approaches = [simple_loop, iterrows_loop, loop_col_first, apply_lambda, apply_predefined, pd_vectorized, np_vectorized]
dfs = [df, df2, df5, df10, df20, df100]

time_per_approach = {}
for fn in approaches:
    runtimes = []
    for df in dfs:
        runtime = %timeit -o -q fn(df)
        runtimes.append(runtime.average)
    time_per_approach.update({fn.__name__: runtimes})


In [ ]:
x = [df.shape[0] for df in dfs]

fig, axes = plt.subplots(
    ncols = 2,
    sharey = True,
    figsize = (15, 5)
)

for label, runtimes in time_per_approach.items():
    for ax in axes:
        ax.plot(
            x,
            runtimes,
            label = label,
            marker = 'o'
        )

axes[0].legend()
axes[1].set_xscale('log')

fig.suptitle('Runtimes of different dataframe mapping approaches', weight='bold')
fig.supxlabel('rows in dataframe')
fig.supylabel('runtime (s)')
fig.tight_layout(w_pad=.5)
fig.savefig('plots/runtimes.png', dpi=500)


We see that at least all non-vectorized approaches scale linearly.
From this figure it is hard to draw any meaningful conclusion on the scaling of the two most efficient approaches (`pd_vectorized` & `np_vectorized`) as the increments are too small.
Another interesting thing to explore would be to see which of the two "apply" methods is most efficient, even though the difference seems trivial.


First we will examine the "vectorized" methods in isolation, and plotting them against a standard linear x axis.


In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

time_per_vectorized_approach = {va: time_per_approach[va] for va in ['pd_vectorized', 'np_vectorized']}

for label, runtimes in time_per_vectorized_approach.items():
    ax.plot(
        x,
        runtimes,
        label=label,
        marker='o'
    )

ax.legend()
ax.set_title('Runtime comparison for vectorized approaches', weight='bold')
ax.set_xlabel('rows in dataframe')
ax.set_ylabel('runtime (s)')
fig.savefig('plots/runtimes_v.png', dpi=500)


It appears that this approach too, scales linearly.
We also see that using the numpy arrays extracted from the pandas series objects is indeed more efficient than using the series objects directly.


We now do a similar thing for the "apply" approaches, i.e. the approaches where we use the dataframe's `apply` method.


In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

time_per_vectorized_approach = {va: time_per_approach[va] for va in ['apply_lambda', 'apply_predefined']}

for label, runtimes in time_per_vectorized_approach.items():
    ax.plot(
        x,
        runtimes,
        label=label,
        marker='o'
    )

ax.legend()
ax.set_title('Runtime comparison for "apply" approaches', weight='bold')
ax.set_xlabel('rows in dataframe')
ax.set_ylabel('runtime (s)')
fig.savefig('plots/runtimes_a.png', dpi=500)


This still does not tell us much, so now we will plot only the differences.


In [ ]:
time_per_vectorized_approach = {va: time_per_approach[va] for va in ['apply_lambda', 'apply_predefined']}

if sum(time_per_approach['apply_predefined']) < sum(time_per_approach['apply_lambda']):
    best, not_best = 'apply_predefined', 'apply_lambda'
else:
    best, not_best = 'apply_lambda', 'apply_predefined'

diffs = (np.array(time_per_approach[not_best]) - np.array(time_per_approach[best])) * 1000

fig, ax = plt.subplots(figsize=(8, 5))

ax.plot(
    x,
    diffs,
    marker='o'
)

ax.set_title(f'Overhead introduced by using "{not_best}"', weight='bold')
ax.set_xlabel('rows in dataframe')
ax.set_ylabel('overhead (ms)')
fig.savefig('plots/runtimes_d.png', dpi=500)


There is some interesting scaling going here, specifically the peak at ±35000 entries.
This could however just be attributed to a margin of error, as the differences are never more than 10 milliseconds.


### Reflection
Consider the following questions:
* What steps can you take to ensure your functions can be vectorized?
* If your function is too hard to vectorize, how can you best loop over your dataframe?
* How can you track down performance bottlenecks in your code?

### References

#### Development Containers
- [Beginner's Series to Dev Containers](https://www.youtube.com/watch?v=61M2takIKl8&list=PLj6YeMhvp2S5G_X6ZyMc8gfXPMFPg3O31) on Youtube

#### Numpy & Pandas
* [Cloud X Lab - introduction to Numpy and Pandas](https://cloudxlab.com/blog/numpy-pandas-introduction/)
* [Zero With Dot - Performance of numpy and pandas - comparison](https://zerowithdot.com/python-numpy-and-pandas-performance/)
* [Sofia Heisler - A Beginner’s Guide to Optimizing Pandas Code for Speed](https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6) basic optimization techniques, see also her PyCon 2017 talk on [YouTube](https://www.youtube.com/watch?v=HN5d490_KKk).
* [Pandas - Enhancing Performance](https://pandas.pydata.org/pandas-docs/stable/user_guide/enhancingperf.html) deeper enhancement techniques
* [Numexpr](https://github.com/pydata/numexpr) easily speed up more complex numpy operations
* [Real Python - Look Ma, No For-Loops: Array Programming With NumPy](https://realpython.com/numpy-array-programming/) a somewhat easier introduction to vectorization
* [Python Like You Mean It - Vectorized Operations](https://www.pythonlikeyoumeanit.com/Module3_IntroducingNumpy/VectorizedOperations.html) a bit more advanced text about vectorization

#### Matplotlib
* [Real Python - Python Plotting With Matplotlib (Guide)](https://realpython.com/python-matplotlib-guide/)
* [Practical Business Python - Effectively Using Matplotlib](https://pbpython.com/effective-matplotlib.html)